In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import utils
from tqdm.auto import tqdm
import tensorflow as tf
#tf.enable_eager_execution()
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior() 

In [1]:
class CVAE(object):
    """Conditional Variational Auto Encoder (CVAE)."""

    def __init__(self, n_latent, n_hidden=50, alpha=0.2):
        self.n_latent = n_latent
        self.n_hidden = n_hidden
        self.alpha = alpha

    def lrelu(self, x, alpha=0.3):
        return tf.maximum(x, tf.multiply(x, alpha))

    def encoder(self, X_in, cond, input_dim):
        with tf.variable_scope("encoder", reuse=None):
            x = tf.concat([X_in, cond], axis=1)
            x = tf.contrib.layers.flatten(x)
            x = tf.layers.dense(x, units=self.n_hidden, activation=self.lrelu)
            mn = tf.layers.dense(x, units=self.n_latent, activation=self.lrelu)
            sd = tf.layers.dense(x, units=self.n_latent, activation=self.lrelu)
            epsilon = tf.random_normal(tf.stack([tf.shape(x)[0], self.n_latent]))
            z = mn + tf.multiply(epsilon, tf.exp(sd / 2.))
            
            return z, mn, sd
    
    def decoder(self, sampled_z, cond, input_dim):
        with tf.variable_scope("decoder", reuse=None):
            x = tf.concat([sampled_z, cond], axis=1)
            x = tf.layers.dense(x, units=self.n_hidden, activation=self.lrelu)
            x = tf.layers.dense(x, units=input_dim, activation=tf.nn.sigmoid)
            x = tf.reshape(x, shape=[-1, input_dim])

            return x

    def train(self, data, data_cond, n_epochs=10000, learning_rate=0.005,
              show_progress=False):
    
        data = utils.as_float_array(data)
        data_cond = utils.as_float_array(data_cond)

        if len(data_cond.shape) == 1:
            data_cond = data_cond.reshape(-1, 1)
    
        assert data.max() <= 1. and data.min() >=0., \
            "All features of the dataset must be between 0 and 1."

        #tf.compat.v1.reset_default_graph()
        tf.reset_default_graph()
        
        #data = data.reshape(len(data),1)
        
        input_dim = data.shape[1]
        dim_cond = data_cond.shape[1]
                

#         X_in = tf.Variable(tf.ones(shape=[None, input_dim]), dtype=tf.float32, name="X")
#         self.cond = tf.Variable(tf.ones(shape=[None, dim_cond]), dtype=tf.float32, name="c")
#         Y = tf.Variable(tf.ones(shape=[None, input_dim]), dtype=tf.float32, name="Y")

        X_in = tf.placeholder(dtype=tf.float32, shape=[None, input_dim],
                              name="X")
                            
        self.cond = tf.placeholder(dtype=tf.float32, shape=[None, dim_cond],
                                   name="c")
        Y = tf.placeholder(dtype=tf.float32, shape=[None, input_dim],
                           name="Y")

        Y_flat = Y

        self.sampled, mn, sd = self.encoder(X_in, self.cond, input_dim=input_dim)           

        self.dec = self.decoder(self.sampled, self.cond, input_dim=input_dim)

        unreshaped = tf.reshape(self.dec, [-1, input_dim])
        decoded_loss = tf.reduce_sum(tf.squared_difference(unreshaped, Y_flat), 1)
        latent_loss = -0.5 * tf.reduce_sum(1. + sd - tf.square(mn) - tf.exp(sd), 1)

        self.loss = tf.reduce_mean((1 - self.alpha) * decoded_loss + self.alpha * latent_loss)

        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

        #########START OF LATENT EXPLORATION#####################
#         print(self.sampled)
# #         print(data)
#         latent_plot_var = self.sess.run(self.sampled, feed_dict={X_in: data, self.cond: data_cond})
#         x_vals = [latent_plot_var[i][0] for i in range(len(latent_plot_var))]
#         y_vals = [latent_plot_var[i][1] for i in range(len(latent_plot_var))]

#         #z_vals = [latent_plot_var[i][2] for i in range(len(latent_plot_var))]
#         color = [0 for i in range(len(x_vals)//2)]
#         color.extend([1 for i in range(len(x_vals)//2)])
        

#         from sklearn import svm
#         from sklearn.metrics import accuracy_score

#         # fit the model
#         clf = svm.SVC(kernel='linear')
#         clf.fit(latent_plot_var, color)
        
#         z = lambda x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x-clf.coef_[0][1]*y) / clf.coef_[0][2]
#         tmp = np.linspace(-2,2,51)
#         x,y = np.meshgrid(tmp,tmp)

        
#         from mpl_toolkits.mplot3d import Axes3D
        
#         fig = plt.figure()
#         ax = Axes3D(fig)
#         ax.plot_surface(x, y, z(x,y))
#         ax.scatter(x_vals, y_vals, z_vals, c=color)

#         plt.show()
        
#         y_pred = clf.predict(latent_plot_var)
#         print(accuracy_score(color, y_pred))

#         mean_x = np.mean(x_vals)
#         mean_y = np.mean(y_vals)
#         max_distance = max(np.sqrt((x_vals - mean_x)**2 + (y_vals - mean_y)**2))
#         threshold = max_distance*0.1
#         accepted_xvals = []
#         accepted_yvals = []
#         for i in range(len(x_vals)):
#             if np.sqrt((x_vals[i] - mean_x)**2 + (y_vals[i] - mean_y)**2) < threshold:
#                 accepted_xvals.append(x_vals[i])
#                 accepted_yvals.append(y_vals[i])
#         print(len(x_vals))
#         print(len(accepted_xvals))
#         plt.scatter(accepted_xvals, accepted_yvals)
#         plt.show()
        
        #########END OF LATENT EXPLORATION#######################
        
        for i in tqdm(range(n_epochs), desc="Training"):
            self.sess.run(optimizer, feed_dict={X_in: data, self.cond: data_cond, Y: data})
            if not i % 1000 and show_progress:
                ls, d = self.sess.run([self.loss, self.dec], feed_dict={X_in: data, self.cond: data_cond, Y: data})

                projections = np.random.randint(0, data.shape[1], size=2)

                plt.scatter(data[:, projections[0]], data[:, projections[1]])
                #               plt.scatter(d[:, projections[0]], d[:, projections[1]])
                plt.show()
                
                print(i, ls)
            
    def generate(self, cond, n_samples=None, mean=[0,0], var=1):
        cond = utils.as_float_array(cond)
        #cond = [cond]
        if n_samples is not None:
            randoms = np.random.normal(0, 1, size=(n_samples, self.n_latent))
            #randoms = np.random.multivariate_normal(mean, cov=np.eye(2), size=n_samples)
            cond = [list(cond)] * n_samples
        else:
            randoms = np.random.normal(mean, var, size=(1, self.n_latent))
            cond = [list(cond)]
            
#         cond = np.array(cond)
#         cond = cond.reshape(len(cond), 1)

        samples = self.sess.run(self.dec, feed_dict={self.sampled: randoms, self.cond: cond})

        if n_samples is None:
            return samples[0]
        
        return samples